In [1]:
from flask import Flask, render_template, request, jsonify
from flask_socketio import SocketIO

In [25]:
app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")  # Enable WebSockets

latex_equation = None
disciplines_data = {}

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/process_latex', methods=['POST'])
def process_latex():
    global latex_equation
    try:
        data = request.get_json()
        latex_equation = data.get('latex', '')
        print(f"Received LaTeX equation: {latex_equation}")
        return jsonify({'status': 'success'})
    except Exception as e:
        return jsonify({'error': str(e)}), 400

@app.route('/get_latex_eq', methods=['GET'])
def get_latex_eq():
    global latex_equation
    return jsonify({'latex_equation': latex_equation})

@app.route('/clear_disciplines', methods=['POST'])
def clear_disciplines():
    global disciplines_data
    disciplines_data = {}  # Reset stored data
    print("Disciplines data has been cleared.")
    return jsonify({'status': 'success', 'message': 'Disciplines cleared successfully!'})

@socketio.on('update_disciplines')
def handle_update_disciplines(data):
    global disciplines_data
    disciplines_data = data
    print(f"Received disciplines data: {disciplines_data}")

    # Emit updated chart data to all connected clients
    socketio.emit('update_chart', disciplines_data)

@app.route('/get_disciplines', methods=['GET'])
def get_disciplines():
    global disciplines_data
    return jsonify(disciplines_data)

@socketio.on('connect')
def handle_connect():
    print("Client connected.")

@socketio.on('disconnect')
def handle_disconnect():
    print("Client disconnected.")

if __name__ == "__main__":
    socketio.run(app, debug=True, host='127.0.0.1', port=5000, use_reloader=False)

127.0.0.1 - - [2025-02-13 14:44:17] "GET / HTTP/1.1" 200 10396 0.012983
127.0.0.1 - - [2025-02-13 14:44:17] "GET /socket.io/?EIO=4&transport=websocket&sid=O-UghEcUNNKiHlruAAAA HTTP/1.1" 200 - 138.625434
127.0.0.1 - - [2025-02-13 14:44:17] "GET /static/css/mathquill.css HTTP/1.1" 304 183 0.009596
127.0.0.1 - - [2025-02-13 14:44:17] "GET /static/js/mathquill.js HTTP/1.1" 304 181 0.000804
127.0.0.1 - - [2025-02-13 14:44:17] "GET /get_disciplines HTTP/1.1" 200 110 0.000844
127.0.0.1 - - [2025-02-13 14:44:17] "GET /socket.io/?EIO=4&transport=polling&t=t5nidovl HTTP/1.1" 200 276 0.001761
127.0.0.1 - - [2025-02-13 14:44:17] "GET /static/css/font/Symbola.woff2 HTTP/1.1" 404 331 0.000613
127.0.0.1 - - [2025-02-13 14:44:17] "POST /socket.io/?EIO=4&transport=polling&t=t5niptse&sid=qj8emh90OKud48CAAAAA HTTP/1.1" 200 195 0.001227
127.0.0.1 - - [2025-02-13 14:44:17] "GET /socket.io/?EIO=4&transport=polling&t=t5niquwt&sid=qj8emh90OKud48CAAAAA HTTP/1.1" 200 189 0.001055
127.0.0.1 - - [2025-02-13 14:44

Client disconnected.
Client connected.


127.0.0.1 - - [2025-02-13 14:44:22] "GET /socket.io/?transport=polling&EIO=4&t=1739479462.559586 HTTP/1.1" 200 276 0.007771


Client connected.
Received disciplines data: {'Computer Science': [], 'Economics': [], 'Mathematics': ['Combinatorial_species.html', 'Fredholm_operator.html'], 'Physics': [], 'Electrical Engineering and System Science': ['Fresnel_integral.html'], 'Quantitative Biology': [], 'Quantitative Finance': [], 'Statistics': []}
Client disconnected.


127.0.0.1 - - [2025-02-13 14:44:23] "GET /socket.io/?transport=websocket&EIO=4&sid=78jU6feZz_GyZxGgAAAC&t=1739479462.587452 HTTP/1.1" 200 208 1.033420
<gevent._socket3.socket at 0x7f9a00221580 object, fd=111, family=2, type=1, proto=0>: (from ('127.0.0.1', 56920)) Invalid HTTP method: '\x88\x82]%Ý-^Í'
127.0.0.1 - - [2025-02-13 14:44:23] "]%Ý-^Í" 200 - 0.000686


Disciplines data has been cleared.


127.0.0.1 - - [2025-02-13 14:44:34] "POST /clear_disciplines HTTP/1.1" 200 184 0.006561
127.0.0.1 - - [2025-02-13 14:44:36] "GET / HTTP/1.1" 200 10396 0.002450
127.0.0.1 - - [2025-02-13 14:44:36] "GET /socket.io/?EIO=4&transport=websocket&sid=qj8emh90OKud48CAAAAA HTTP/1.1" 200 - 18.858156
127.0.0.1 - - [2025-02-13 14:44:36] "GET /static/css/mathquill.css HTTP/1.1" 304 183 0.013121
127.0.0.1 - - [2025-02-13 14:44:36] "GET /static/js/mathquill.js HTTP/1.1" 304 181 0.001831
127.0.0.1 - - [2025-02-13 14:44:36] "GET /socket.io/?EIO=4&transport=polling&t=t624c448 HTTP/1.1" 200 276 0.003565
127.0.0.1 - - [2025-02-13 14:44:36] "GET /get_disciplines HTTP/1.1" 200 110 0.003522
127.0.0.1 - - [2025-02-13 14:44:36] "GET /static/css/font/Symbola.woff2 HTTP/1.1" 404 331 0.002184
127.0.0.1 - - [2025-02-13 14:44:36] "POST /socket.io/?EIO=4&transport=polling&t=t624wema&sid=2dQBesWliF9P8biKAAAE HTTP/1.1" 200 195 0.002097
127.0.0.1 - - [2025-02-13 14:44:36] "GET /socket.io/?EIO=4&transport=polling&t=t624w

Client disconnected.
Client connected.


KeyboardInterrupt
2025-02-13T20:44:43Z


KeyboardInterrupt: 